In [1]:
import numpy as np
import pandas as pd

top_2017 = pd.read_csv('../datasets/top2017.csv')
top_2018 = pd.read_csv('../datasets/top2018.csv')
top_2019 = pd.read_csv('../datasets/top2019.csv', encoding='mbcs', index_col=0)
top_2010_2019 = pd.read_csv('../datasets/top2010-2019.csv', encoding='mbcs', index_col=0)


In [63]:
top_2010_2019.head()

,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [29]:
title = top_2010_2019["title"]
print(len(pd.unique(title)))

584


In [30]:
artists = top_2010_2019["artist"]
print(len(pd.unique(artists)))

184


In [31]:
genres = top_2010_2019["top genre"]
print(len(pd.unique(genres)))

50


In [52]:
#YEAR
year = top_2010_2019["year"]
pd.unique(year)

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
      dtype=int64)

In [64]:
#BPM
bpm = top_2010_2019["bpm"]
print("Num Rows: " + str(len(bpm)))
print("Min Value: " + str(bpm.min()))
print("Max Value: " + str(bpm.max()))
Q1 = bpm.quantile(0.25)
Q3 = bpm.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(bpm < Q1 - (1.5 * IQR)) | (bpm > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 206


(28, 14)

In [67]:
#ENRGY
energy = top_2010_2019["nrgy"]
print("Num Rows: " + str(len(energy)))
print("Min Value: " + str(energy.min()))
print("Max Value: " + str(energy.max()))
Q1 = energy.quantile(0.25)
Q3 = energy.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(energy < Q1 - (1.5 * IQR)) | (energy > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 98


(11, 14)

In [71]:
#DANCE
dance = top_2010_2019["dnce"]
print("Num Rows: " + str(len(dance)))
print("Min Value: " + str(dance.min()))
print("Max Value: " + str(dance.max()))
Q1 = dance.quantile(0.25)
Q3 = dance.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(dance < Q1 - (1.5 * IQR)) | (dance > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 97


(17, 14)

In [72]:
#DB
decibles = top_2010_2019["dB"]
print("Num Rows: " + str(len(decibles)))
print("Min Value: " + str(decibles.min()))
print("Max Value: " + str(decibles.max()))
Q1 = decibles.quantile(0.25)
Q3 = decibles.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(decibles < Q1 - (1.5 * IQR)) | (decibles > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: -60
Max Value: -2


(15, 14)

In [73]:
#LIVE
live = top_2010_2019["live"]
print("Num Rows: " + str(len(live)))
print("Min Value: " + str(live.min()))
print("Max Value: " + str(live.max()))
Q1 = live.quantile(0.25)
Q3 = live.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(live < Q1 - (1.5 * IQR)) | (live > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 74


(23, 14)

In [74]:
#VAL
valence = top_2010_2019["val"]
print("Num Rows: " + str(len(valence)))
print("Min Value: " + str(valence.min()))
print("Max Value: " + str(valence.max()))
Q1 = valence.quantile(0.25)
Q3 = valence.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(valence < Q1 - (1.5 * IQR)) | (valence > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 98


(0, 14)

In [77]:
#DUR
duration = top_2010_2019["dur"]
print("Num Rows: " + str(len(duration)))
print("Min Value: " + str(duration.min()))
print("Max Value: " + str(duration.max()))
Q1 = duration.quantile(0.25)
Q3 = duration.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(duration < Q1 - (1.5 * IQR)) | (duration > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 134
Max Value: 424


(19, 14)

In [78]:
#ACOUS
acoustic = top_2010_2019["acous"]
print("Num Rows: " + str(len(acoustic)))
print("Min Value: " + str(acoustic.min()))
print("Max Value: " + str(acoustic.max()))
Q1 = acoustic.quantile(0.25)
Q3 = acoustic.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(acoustic < Q1 - (1.5 * IQR)) | (acoustic > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 99


(66, 14)

In [79]:
#SPCH
speech = top_2010_2019["spch"]
print("Num Rows: " + str(len(speech)))
print("Min Value: " + str(speech.min()))
print("Max Value: " + str(speech.max()))
Q1 = speech.quantile(0.25)
Q3 = speech.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(speech < Q1 - (1.5 * IQR)) | (speech > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 48


(66, 14)

In [80]:
#POP
popular = top_2010_2019["pop"]
print("Num Rows: " + str(len(popular)))
print("Min Value: " + str(popular.min()))
print("Max Value: " + str(popular.max()))
Q1 = popular.quantile(0.25)
Q3 = popular.quantile(0.75)
IQR = Q3 - Q1
top_2010_2019.loc[(popular < Q1 - (1.5 * IQR)) | (popular > Q3 + (1.5 * IQR))].shape

Num Rows: 603
Min Value: 0
Max Value: 99


(21, 14)